In [1]:
# ===========================
# p16_ensemble_refine.ipynb
# Refinamiento de ensembles sobre features de p11/p14
# ===========================
import os, json, math, itertools
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, precision_score, recall_score

# ---------------------------
# A) Rutas y carga de datos
# ---------------------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

DRIVE = Path("/content/drive/MyDrive/CognitivaAI")
P11 = DRIVE / "p11_alt_backbones"
P14 = DRIVE / "p14_oasis2_images"   # (sólo referencia; los features consolidados están en P11)
OUT = DRIVE / "p16_ensemble_refine"
OUT.mkdir(parents=True, exist_ok=True)

VAL_PATH  = P11 / "val_patient_features_backbones.csv"
TEST_PATH = P11 / "test_patient_features_backbones.csv"

val = pd.read_csv(VAL_PATH)
test = pd.read_csv(TEST_PATH)

print("VAL:", val.shape, "| cols:", len(val.columns))
print("TEST:", test.shape, "| cols:", len(test.columns))

# ---------------------------
# B) Cohortes y columnas útiles
# ---------------------------
def tag_cohort(pid: str) -> str:
    return "OAS2" if str(pid).startswith("OAS2_") else "OAS1"

for df in (val, test):
    df["cohort"] = df["patient_id"].astype(str).map(tag_cohort)

# Identificar columnas de features candidatas: agregados por slice
def is_feat(c):
    if c in ["patient_id", "y_true", "cohort"]:
        return False
    # típicamente vienen *_mean, *_trimmed20, *_top7, *_p2
    tail_ok = any(c.endswith(suf) for suf in ["_mean","_trimmed20","_top7","_p2"])
    return tail_ok

feat_cols_all = [c for c in val.columns if is_feat(c)]
print("Total feats detectadas:", len(feat_cols_all))

# Opcional: filtrar por ratio de NaN aceptable mirando VAL (para no sobre-filtrar TEST)
nan_ratio = val[feat_cols_all].isna().mean().sort_values(ascending=False)
print("\nNaN ratio (VAL) top 10:\n", nan_ratio.head(10))
keep_cols = nan_ratio[nan_ratio <= 0.4].index.tolist()
drop_cols = [c for c in feat_cols_all if c not in keep_cols]
print(f"\n✅ Mantengo {len(keep_cols)} columnas; ❌ descarto por NaN>0.4: {len(drop_cols)}")

X_val = val[keep_cols].copy()
y_val = val["y_true"].astype(int).values
X_test = test[keep_cols].copy()
y_test = test["y_true"].astype(int).values

# ---------------------------
# C) Métricas
# ---------------------------
def metrics_from_scores(y_true, y_score, thr=0.5):
    # AUC / PRAUC robustos si hay ambas clases
    try:
        auc = roc_auc_score(y_true, y_score)
    except Exception:
        auc = np.nan
    try:
        prauc = average_precision_score(y_true, y_score)
    except Exception:
        prauc = np.nan
    y_pred = (y_score >= thr).astype(int)
    acc = accuracy_score(y_true, y_pred)
    P = precision_score(y_true, y_pred, zero_division=0)
    R = recall_score(y_true, y_pred, zero_division=0)
    return dict(AUC=auc, PRAUC=prauc, Acc=acc, P=P, R=R, thr=thr, n=len(y_true))

def eval_by_cohort(df_pred, score_col="y_score", thr=0.5):
    out = {}
    for k, g in df_pred.groupby("cohort"):
        out[k] = metrics_from_scores(g["y_true"].values, g[score_col].values, thr=thr)
    out["ALL"] = metrics_from_scores(df_pred["y_true"].values, df_pred[score_col].values, thr=thr)
    return out

# ---------------------------
# D) Meta-modelo 1: LR (imputación+scaler)
# ---------------------------
num_cols = keep_cols

pre_lr = ColumnTransformer(
    transformers=[
        ("imputer", SimpleImputer(strategy="median", add_indicator=True), num_cols),
        ("scaler", StandardScaler(with_mean=False), slice(0, 0))  # placeholder; escalamos dentro del pipeline
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

# Nota: MissingIndicator ya lo añade SimpleImputer con add_indicator=True
pipe_lr = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
    ("scaler", StandardScaler(with_mean=False)),
    ("clf", LogisticRegression(solver="lbfgs", max_iter=2000, class_weight=None))
])

pipe_lr.fit(X_val, y_val)
p_val_lr  = pipe_lr.predict_proba(X_val)[:,1]
p_test_lr = pipe_lr.predict_proba(X_test)[:,1]

# ---------------------------
# E) Meta-modelo 2: HGB (acepta NaNs) + calibración en VAL
# ---------------------------
hgb = HistGradientBoostingClassifier(
    loss="log_loss",
    max_depth=None,
    learning_rate=0.06,
    max_iter=600,
    min_samples_leaf=10,
    l2_regularization=1e-3,
    random_state=42
)
# Calibramos con CV interno en VAL (sigmoid tipo Platt)
cal_hgb = CalibratedClassifierCV(hgb, method="sigmoid", cv=5)
cal_hgb.fit(X_val, y_val)

p_val_hgb  = cal_hgb.predict_proba(X_val)[:,1]
p_test_hgb = cal_hgb.predict_proba(X_test)[:,1]

# ---------------------------
# F) Blending simple LR-HGB: buscar α en VAL que maximiza AUC
# ---------------------------
alphas = np.linspace(0, 1, 51)  # 0.00..1.00
best = (-np.inf, 0.5)  # (AUC, alpha)
for a in alphas:
    p_blend = a*p_val_lr + (1-a)*p_val_hgb
    try:
        auc = roc_auc_score(y_val, p_blend)
    except Exception:
        auc = -np.inf
    if auc > best[0]:
        best = (auc, a)

alpha_opt = float(best[1])
print(f"\n⭐ Blending óptimo en VAL: alpha={alpha_opt:.2f} (LR weight) | AUC={best[0]:.4f}")

p_val_blend  = alpha_opt*p_val_lr + (1-alpha_opt)*p_val_hgb
p_test_blend = alpha_opt*p_test_lr + (1-alpha_opt)*p_test_hgb

# ---------------------------
# G) Búsqueda simple de umbral en VAL para maximizar F1 o balance P/R
# ---------------------------
def find_best_thr(y_true, y_score, grid=np.linspace(0.05,0.95,19)):
    best = (0, 0.5)
    for t in grid:
        y_pred = (y_score>=t).astype(int)
        P = precision_score(y_true, y_pred, zero_division=0)
        R = recall_score(y_true, y_pred, zero_division=0)
        if (P+R)>0:
            f1 = 2*P*R/(P+R)
        else:
            f1 = 0
        if f1 > best[0]:
            best = (f1, float(t))
    return {"thr": best[1], "F1": best[0]}

thr_lr    = find_best_thr(y_val, p_val_lr )["thr"]
thr_hgb   = find_best_thr(y_val, p_val_hgb)["thr"]
thr_blend = find_best_thr(y_val, p_val_blend)["thr"]
print(f"Umbrales (VAL) ~F1max → LR:{thr_lr:.2f} | HGB:{thr_hgb:.2f} | Blend:{thr_blend:.2f}")

# ---------------------------
# H) Evaluación y guardado
# ---------------------------
pred_val = pd.DataFrame({
    "patient_id": val["patient_id"],
    "cohort": val["cohort"],
    "y_true": y_val,
    "p_lr": p_val_lr,
    "p_hgb": p_val_hgb,
    "p_blend": p_val_blend
})
pred_test = pd.DataFrame({
    "patient_id": test["patient_id"],
    "cohort": test["cohort"],
    "y_true": y_test,
    "p_lr": p_test_lr,
    "p_hgb": p_test_hgb,
    "p_blend": p_test_blend
})

# Métricas por modelo/conjunto
summary = {
    "alpha_opt": alpha_opt,
    "VAL": {
        "LR":    eval_by_cohort(pred_val.rename(columns={"p_lr":"y_score"}),  "y_score", thr=thr_lr),
        "HGB":   eval_by_cohort(pred_val.rename(columns={"p_hgb":"y_score"}), "y_score", thr=thr_hgb),
        "BLEND": eval_by_cohort(pred_val.rename(columns={"p_blend":"y_score"}),"y_score", thr=thr_blend),
    },
    "TEST": {
        "LR":    eval_by_cohort(pred_test.rename(columns={"p_lr":"y_score"}),  "y_score", thr=thr_lr),
        "HGB":   eval_by_cohort(pred_test.rename(columns={"p_hgb":"y_score"}), "y_score", thr=thr_hgb),
        "BLEND": eval_by_cohort(pred_test.rename(columns={"p_blend":"y_score"}),"y_score", thr=thr_blend),
    },
    "thresholds": {"VAL_LR":thr_lr, "VAL_HGB":thr_hgb, "VAL_BLEND":thr_blend}
}

# Guardados
pred_val.to_csv(OUT / "p16_val_patient_preds_ensemble.csv", index=False)
pred_test.to_csv(OUT / "p16_test_patient_preds_ensemble.csv", index=False)
with open(OUT / "p16_ensemble_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\n💾 Guardados:")
print(" -", OUT / "p16_val_patient_preds_ensemble.csv")
print(" -", OUT / "p16_test_patient_preds_ensemble.csv")
print(" -", OUT / "p16_ensemble_summary.json")

# Vista rápida de métricas clave
def pretty_block(name, d):
    def fmt(m):
        return {k:(round(v,4) if isinstance(v,(int,float,np.floating)) and not math.isnan(v) else v) for k,v in m.items()}
    print(f"\n[{name}]")
    for split in ["VAL","TEST"]:
        for model in ["LR","HGB","BLEND"]:
            coh = d[split][model]
            txt = " | ".join(
                f"{ck}:{fmt(coh[ck])}" for ck in ["OAS1","OAS2","ALL"] if ck in coh
            )
            print(f"{split}-{model}: {txt}")

pretty_block("Resumen", summary)


Mounted at /content/drive
VAL: (69, 58) | cols: 58
TEST: (70, 58) | cols: 58
Total feats detectadas: 56

NaN ratio (VAL) top 10:
 patient_preds_ensemble_trimmed20    0.855072
patient_preds_ensemble_top7         0.855072
patient_preds_ensemble_p2           0.855072
patient_preds_ensemble_mean         0.855072
patient_preds_trimmed20             0.855072
patient_preds_top7                  0.855072
patient_preds_p2                    0.855072
patient_preds_mean                  0.855072
slices_preds_mean                   0.855072
slices_preds_p2                     0.855072
dtype: float64

✅ Mantengo 36 columnas; ❌ descarto por NaN>0.4: 20


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



⭐ Blending óptimo en VAL: alpha=0.02 (LR weight) | AUC=0.9533
Umbrales (VAL) ~F1max → LR:0.45 | HGB:0.45 | Blend:0.45

💾 Guardados:
 - /content/drive/MyDrive/CognitivaAI/p16_ensemble_refine/p16_val_patient_preds_ensemble.csv
 - /content/drive/MyDrive/CognitivaAI/p16_ensemble_refine/p16_test_patient_preds_ensemble.csv
 - /content/drive/MyDrive/CognitivaAI/p16_ensemble_refine/p16_ensemble_summary.json

[Resumen]
VAL-LR: OAS1:{'AUC': np.float64(0.9444), 'PRAUC': np.float64(0.9525), 'Acc': 0.8936, 'P': 0.8571, 'R': 0.9, 'thr': 0.45, 'n': 47} | OAS2:{'AUC': np.float64(0.5455), 'PRAUC': np.float64(0.5238), 'Acc': 0.5, 'P': 0.5, 'R': 1.0, 'thr': 0.45, 'n': 22} | ALL:{'AUC': np.float64(0.8812), 'PRAUC': np.float64(0.8564), 'Acc': 0.7681, 'P': 0.6744, 'R': 0.9355, 'thr': 0.45, 'n': 69}
VAL-HGB: OAS1:{'AUC': np.float64(1.0), 'PRAUC': np.float64(1.0), 'Acc': 1.0, 'P': 1.0, 'R': 1.0, 'thr': 0.45, 'n': 47} | OAS2:{'AUC': np.float64(0.5), 'PRAUC': np.float64(0.5), 'Acc': 0.5, 'P': 0.5, 'R': 1.0, 't